In [1]:
from data_processing.data_module import FaceDataModule
from data_processing.dataset import FaceDataset
from model.gauss_predictor import GaussPredictor
import yaml
import k3d
from gaussian_splatting.utils.sh_utils import SH2RGB
import torch
from tqdm import tqdm
from torchmetrics.image import PeakSignalNoiseRatio

config = yaml.safe_load(open('configs/settings_final.yaml', 'r'))
config['data']['force_rebuild'] = False
config['data']['save_results'] = False

face_data_module = FaceDataModule(config=config)
face_data_module.setup('fit')

face_data_loader_val = face_data_module.val_dataloader()

gauss_predictor = GaussPredictor.load_from_checkpoint(config['model']['checkpoint'], config=config, data_module=face_data_module)
gauss_predictor = gauss_predictor.cpu()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


/home/oroz/anaconda3/envs/gaussian_splatting/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/oroz/anaconda3/envs/gaussian_splatting/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


ENCODER: F
train self.face_ids ['017', '018', '024', '030', '031', '032', '033', '035', '036', '037', '038', '040', '042', '043', '055', '056', '057', '058', '060', '061', '063', '064', '065', '067', '068', '068', '069', '071', '072', '073', '074', '075', '076', '077', '078', '079', '080', '081', '082', '083', '085', '088', '089', '090', '091', '092', '093', '094', '095', '096', '097', '098', '099', '100', '102', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '162', '163', '164', '165', '166', '167', '168', '170', '171', '172', '173', '174', '175', '176', '177', '178', '179', '180', '181', '182', '183', '184', '185', '186', '187', '188', '189', '191', '192', '193', '194', '195', '197', '198', '199', '200', '201', '2

/home/oroz/anaconda3/envs/gaussian_splatting/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/oroz/anaconda3/envs/gaussian_splatting/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /home/oroz/anaconda3/envs/gaussian_splatting/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth


In [2]:
from statistics import mean


def compute_psnr_loss(rendered_images, target_images):
    losses = []
    for serial, rendered_image in rendered_images.items():
        psnr = PeakSignalNoiseRatio(data_range=1.0).cuda()
        target_image = target_images[serial]
        losses.append(psnr(rendered_image.unsqueeze(0), target_image.unsqueeze(0)).unsqueeze(0))
    
    losses = torch.cat(losses, dim=0).to(gauss_predictor.device)
    return losses
    

ids = []
lpips_scores = []
psnr_scores = []
ssim_scores = []

for batch in tqdm(face_data_loader_val):
    batch = face_data_module.transfer_batch_to_device(batch, 'cpu', 0)
    initial_gaussians = batch['initial_gaussians']
    images = batch['images']
    target_depths = batch['depths']
    sequence_id = batch['sequence_id']
    
    camera_params = batch['camera_params']
    camera_params['cam_2_world_poses'] = FaceDataset.dicts_to_poses(camera_params['cam_2_world_poses'])
    
    input_serials = batch['input_serials']
    input_cam_2_world_poses = {serial: cam_2_world_pose for serial, cam_2_world_pose in camera_params['cam_2_world_poses'].items() if serial in input_serials}
    input_camera_params = {'cam_2_world_poses': input_cam_2_world_poses, 'intrinsics': camera_params['intrinsics']}
    
    gaussian_model = gauss_predictor.get_gaussian_model(sequence_id)
    
    predicted_gaussians = gauss_predictor.forward(initial_gaussians)
    predicted_gaussians = {key: val.cuda() for key, val in predicted_gaussians.items()}
    gaussian_model.set_gaussian_properties(predicted_gaussians)
    
    first_image = list(images.values())[0]
    _, image_height, image_width = first_image.shape
    
    rendered_images, rendered_depth = gauss_predictor.render_images(
        gaussian_model=gaussian_model,
        camera_parameters=camera_params,
        height=image_height,
        width=image_width,
        device='cuda'
    )
    
    rendered_images = {key: val.cpu() for key, val in rendered_images.items()}
    
    lpips_loss = gauss_predictor.losses.compute_lpips_loss('val', rendered_images, images).mean()
    ssim_loss = gauss_predictor.losses.compute_ssim_loss('val', rendered_images, images).mean()
    psnr_loss = compute_psnr_loss(rendered_images, images).mean()
    
    lpips_scores.append(lpips_loss)
    ssim_scores.append(ssim_loss)
    psnr_scores.append(psnr_loss)
    
    id=sequence_id.replace('/EXP-1-head','')
    ids.append(id)
    assert id in gauss_predictor.test_ids
    
    
print(f"{'ID'<10} {'LPIPS':<10} {'SSIM':<10} {'PSNR':<10}")
for i, id in enumerate(ids):
    print(f"{id<10} {lpips_scores[i]:.04f<10} {ssim_scores[i]:.04f<10} {psnr_scores[i]:.04f<10}")

print('')
print(f"{'all'<10} {mean(lpips_scores):.04f<10} {mean(ssim_scores):.04f<10} {mean(psnr_scores):.04f<10}")

 20%|██        | 2/10 [00:23<01:33, 11.68s/it]


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB. GPU 0 has a total capacity of 9.77 GiB of which 2.38 MiB is free. Process 359245 has 6.98 GiB memory in use. Including non-PyTorch memory, this process has 2.76 GiB memory in use. Of the allocated memory 2.38 GiB is allocated by PyTorch, and 84.17 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)